# Part 5

### Problem Statement
Choose one of the topics we will cover in week 6. For your chosen topic, experiment with it and document your experiments, findings and code in a Python Notebook. If you choose model interpretability, then include some text near the
beginning discussing your findings and comparing the results of the different methods. If you choose another model type (such as stacked ensembles or auto ML), then fit the models and document your model fitting process in a similar
way to part 4. 

If, for this part, you have chosen to fit another model, then include a screenshot of your competition entry as usual.

Make sure that your notebook is neat and well commented and uses markdown as appropriate to make it easy to follow.

Some suggestions for you to consider are below:

#### vowpal wabbit

- Adjust the code provided in the notebooks to create data for vw in the correct format.
- Train vw with some reasonable default parameters.
- Make a note of the validation loss.
- Create predictions on test and submit on Kaggle. Compare your Kaggle result to your validation loss. It should be similar. If it is not - it may be hard to understand why but you should at least note the difference.
- There are various ways you could do h-p training. You could turn the above process into a loop and do a grid-search over some of the key parameters (the speed of vw should make this possible). Alternatively there is at least one set of code online which claims to use hyperopt - getting this to work would be impressive.
- How does your best vw model compare to your Part 3 model? Which is better? Why is it better? - after all, they are both penalised logistic regression models.

*Disclaimer: The preprocessing process is the same as that of Part 3 since we are going to compare the two framework

## Set-up

### Import relevant libraries

In [1]:
import os
import re
import csv
import numpy as np
import pandas as pd
import pickle

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder

import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.grid.grid_search import H2OGridSearch

### Set directories and paths

In [2]:
# set directories
print(os.getcwd())
dirRawData = os.path.join('..', 'input')
dirPData   = os.path.join('..', 'PData')
dirPOutput = os.path.join('..', 'POutput')

/home/jovyan/smm284-aml/assignment/PCode


### Load data

In [3]:
# load data: 250k train data
f_name = '01_df_250k.pickle'
f_path = os.path.join(dirPData, f_name)
PData = pickle.load(open(f_path, "rb"))

# separate data into train/test set
train_set = PData['df_train']
test_set = PData['df_test']

In [4]:
# load variable metadata 
f_name = '01_vars.pickle'
f_path = os.path.join(dirPData, f_name)
var_meta = pickle.load(open(f_path, "rb"))

# extract lists from metadata
var_idx_num = var_meta['vars_ind_numeric']
var_idx_cat = var_meta['vars_ind_categorical']
var_idx_hccv = var_meta['vars_ind_hccv']
var_idx_id = var_meta['vars_notToUse']
var_idx_response = var_meta['var_dep']

### Clean missing values
#### - Deal appropriately with missings (for all numeric variables, -99 means missing).

In [5]:
# replace -99 with np.nan
train_set = train_set.replace(-99, np.nan)
test_set = test_set.replace(-99, np.nan)

In [6]:
# count number of nulls in each column -- train_set
np.array(train_set.isnull().sum(axis = 0))

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0, 102984,
        23788,  23788,  23788,   1852,   2090,  23898,  24321,  11673,
        32806,  33764, 197803,   8895,   6151,  55544,  57869,  57136,
            2,      0,      0,      0,      0,      0,      0,   1092,
            0,      0,      0,      0,      0,      0,      0,      1,
            1,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0])

In [7]:
# count number of nulls in each column -- test_set
np.array(test_set.isnull().sum(axis = 0))

array([     0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0, 283601,
            0,      0,      0,      0,      0,      2,      0,      1,
            0,      0,      0,      0,      0,      1,      0,      9,
          226,  10870,      9,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,     17, 131970,  26261,
        26261,  26261,   3429,   3504,  26298,  26356,  18076,   2261,
         2356,   2756,  10130,   6363,  18816,  22520,  20432,      2,
            0,      0,      0,      0,      0,      0,   3560,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0])

Strategy:
- remove the feature if the number of null values in the `test_set` exceed 250
- drop rows that have leftover null values in the `train_set`
- for leftover null values in the `test_set` impute with `mean` for numerical features and `most frequent` for categorical features

Through data exploration, we decided to remove features that have over 250 null values in the test set. That is because imputation can introduce biases to the data as it is impossible to perfectly predict the true values that are missing. Still, it is not possible to remove observations from the test set concerning the competition. We want to preserve as much information as possible and thus, for features with fewer than 250 null values (less than 0.1% of the data) imputation for mean / most frequent for numeric / categorical features is incorporated as imputing small number of observations with these values does not have a large impact on the overall distribution.

For the train set, after removing features with over 250 null values in the test set, very small numbers of null values are observed (5 in total). We decided to remove those observations and thus we only lose 3 observations in total.

In [8]:
# number of null values in the test set exceed 250
to_remove = test_set.columns[np.array(test_set.isnull().sum(axis = 0) > 250)]

In [9]:
# remove the variables
train_set = train_set.drop(columns=to_remove)
test_set = test_set.drop(columns=to_remove)

In [10]:
# display number of leftover missing values in `train_set`
np.array(train_set.isnull().sum(axis = 0))

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [11]:
# drop rows that have leftover null values in the `train_set`
train_set = train_set.dropna(axis=0)

In [12]:
# display number of leftover missing values in `test_set`
np.array(test_set.isnull().sum(axis = 0))

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   2,   0,   1,   0,   0,   0,
         0,   0,   1,   0,   9, 226,   9,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,  17,   2,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])

In [13]:
# get columns in the test_set with missing values
impute_col = test_set.columns[np.array(test_set.isnull().sum(axis = 0) > 0)]
impute_col

Index(['c09', 'e03', 'e24', 'e17', 'e18', 'e20', 'f10', 'e02'], dtype='object')

In [14]:
# display data for columns that are going to impute
to_impute = test_set[impute_col]
to_impute.head()

,c09,e03,e24,e17,e18,e20,f10,e02
0,B,A,A,920BD,11250,A4893,CKG,6.0
1,B,A,A,F33BC,5614E,D53C5,AIR,15.0
2,B,A,A,F33BC,5614E,D53C5,AEL,15.0
3,A,A,A,861C8,9076B,A4893,CUO,27.0
4,A,A,A,861C8,9076B,A4893,AWV,27.0


In [15]:
# get column indices for numerical and categorical features that need imputation
num_col = to_impute.select_dtypes(include='number').columns
cat_col = to_impute.select_dtypes(include='category').columns
# create imputation pipeline
# -- parameters = List of (name, transformer, columns)
pipeline = ColumnTransformer([
    ('cat_impute', SimpleImputer(strategy='most_frequent'), cat_col),
    ('num_impute', SimpleImputer(strategy='mean'), num_col)
])
# impute missing values
imputed_test = pipeline.fit_transform(test_set)

In [16]:
# assign values back to the `test_set`
test_set[impute_col] = imputed_test

Check again the number of missing values

In [17]:
print(f'Number of missing values in `train_set`: {np.sum(np.sum(train_set.isna()))}')
print(f'Number of missing values in `test_set`: {np.sum(np.sum(test_set.isna()))}')

Number of missing values in `train_set`: 0
Number of missing values in `test_set`: 0


As some features were removed, we adjust the metadata

In [18]:
var_idx_num = list(set(var_idx_num) - set(to_remove))
var_idx_cat = list(set(var_idx_cat) - set(to_remove))
var_idx_hccv = list(set(var_idx_hccv) - set(to_remove))

There is also a feature with only 1 unique value: `'e16'`, we removed it from the data.

In [19]:
# column with only 1 unique value
train_set.columns[(train_set.nunique() == 1)]

Index(['e16'], dtype='object')

In [20]:
# remove from the dataframes
train_set = train_set.drop(columns='e16')
test_set = test_set.drop(columns='e16')

# update metadata
var_idx_num.remove('e16')

### Feature engineering

#### - Deal with numerics - i.e. for at least some try linear splines (or another method of your choice to deal with non-linear effects).
    - Apply linear splines to numeric features with more than 20 unique values
    - Spline using the percentiles from the train set and applies the same transformation to the test set. 
Similar logic as other preprocessings (e.g. standardisation), the transformation is fitted using the training set and applied to the test. The test set should be kept as unseen (i.e., not knowing the distribution during the training process) and thus the linear spline should be applied base on the distribution of the train set. 

In [21]:
# extract numeric features from train/test sets
df_train_num = train_set[var_idx_num]
df_test_num = test_set[var_idx_num]

In [22]:
# select only features with over 20 unique values to spline
var_to_spline = list(df_train_num.columns[df_train_num.nunique().values > 20])
print('var_to_spline:\n',
      var_to_spline)

var_to_spline:
 ['f02', 'e15', 'e04', 'e02', 'e08', 'e05']


In [23]:
# define a spline function that transform both train/test set
def fn_spline_train_test(var, x_train, x_test, n_spline):
    # define percentile step size
    step = 100 // n_spline
    # get the percentiles from the train set
    ptiles = np.percentile(x_train, np.arange(step, 100+step, step))
    # initialise dataframes
    train_ptiles = pd.DataFrame({var: x_train})
    test_ptiles = pd.DataFrame({var: x_test})
    # spline the variable
    for idx, ptile in enumerate(ptiles):
        train_ptiles[f'{var}_{str(idx)}'] = np.maximum(0, x_train - ptiles[idx])
        test_ptiles[f'{var}_{str(idx)}'] = np.maximum(0, x_test - ptiles[idx])
    return [train_ptiles, test_ptiles]

In [24]:
# applies linear spline to both train/test sets
for var in var_to_spline:
    # generate a dataframe for spline variables
    train_ptiles, test_ptiles = fn_spline_train_test(var=var, 
                                                     x_train=df_train_num[var],
                                                     x_test=df_test_num[var],
                                                     n_spline=5)
    # drop the variable that were transformed
    df_train_num = df_train_num.drop(columns=[var])
    df_test_num = df_test_num.drop(columns=[var])
    # concat the spline variables
    df_train_num = pd.concat([df_train_num, train_ptiles], axis=1, sort=False)
    df_test_num = pd.concat([df_test_num, test_ptiles], axis=1, sort=False)

In [25]:
# display data
df_test_num.head()

,f26,f06,e06,f21,f20,f28,f11,f16,f31,f25,...,e08_1,e08_2,e08_3,e08_4,e05,e05_0,e05_1,e05_2,e05_3,e05_4
0,0,60,20,0,0,5,10,0,0,0,...,0.0,0.0,0.0,0.0,58,39.0,19.0,0.0,0.0,0.0
1,0,60,76,0,0,0,15,0,0,0,...,0.0,0.0,0.0,0.0,76,57.0,37.0,17.0,0.0,0.0
2,0,60,76,0,1,5,8,0,-3,0,...,0.0,0.0,0.0,0.0,76,57.0,37.0,17.0,0.0,0.0
3,0,60,76,0,0,5,13,0,0,0,...,0.0,0.0,0.0,0.0,77,58.0,38.0,18.0,0.0,0.0
4,0,60,76,0,0,5,12,0,0,0,...,0.0,0.0,0.0,0.0,77,58.0,38.0,18.0,0.0,0.0


In [26]:
# create metadata for numeric features after spline
var_idx_spline = df_train_num.columns.tolist()

#### - Deal with hccvs (eg using the feature encoding library that we looked at in lecture) (You do not need to deal with low cardinality categorical features since H2O will one-hot them for you).
For this part, we used Target Encoder to get the mean of the response variable within each class. 

*It performs better than the other encoders.

In [27]:
# extract hccv features from train/test sets
df_train_hccv = train_set[var_idx_hccv]
df_test_hccv = test_set[var_idx_hccv]

In [28]:
# encode hccv with target encoder
enc = TargetEncoder()
# apply target encoder
df_train_hccv = enc.fit_transform(df_train_hccv, train_set[var_idx_response])
df_test_hccv = enc.transform(df_test_hccv)

/opt/conda/lib/python3.9/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


#### - Try out some interactions.
We decided to try applying interaction among 6 top categorical variables based on the feature importances performed in part 2. (We used 6 as there is a gap between 6th and 7th features.

The variables are as follows: `['e21', 'b04', 'f09', 'e20', 'e11', 'a03']`

#### - Try out some other features (eg division of numerics).
We decided to try the division of numeric features. Considering the features we used to apply linear spline, they are deemed to be higher important features with larger varities (i.e., unique values > 20). 

The variables are as follows: `['e04', 'e05', 'e02', 'e08', 'e15', 'f02']`

From the below table, we noticed that many `e04, e05, etc` have quite close distribution (mean and std). As well as `'f02', 'f01'`;
Therefore, we decided to perform division of these pairs:
`[('f01', 'f02'), ('e04', 'e05'), ('e02', 'e15')]`

In [29]:
# display distribution of numeric features
train_set[var_idx_num].describe().T.sort_values(by='mean', ascending=False).head(15)

,count,mean,std,min,25%,50%,75%,max
e09,249997.0,50.191974,27.948693,18.0,18.0,55.0,76.0,99.0
e08,249997.0,50.031268,28.215267,0.0,16.0,53.0,75.0,99.0
e04,249997.0,49.824678,28.819172,1.0,24.0,49.0,76.0,99.0
f02,249997.0,49.641864,28.872189,2.0,25.0,49.0,74.0,99.0
f01,249997.0,49.588539,28.139675,13.0,13.0,51.0,75.0,99.0
e02,249997.0,49.431249,28.883891,0.0,23.0,50.0,75.0,99.0
e15,249997.0,49.427393,28.604578,0.0,22.0,51.0,74.0,99.0
e06,249997.0,49.387841,27.694698,20.0,20.0,49.0,76.0,99.0
f06,249997.0,49.367396,20.585375,0.0,60.0,60.0,60.0,60.0
f08,249997.0,49.187510,9.329284,0.0,51.0,51.0,51.0,51.0


In [30]:
# division features
# initialise dataframes
df_train_div = pd.DataFrame()
df_test_div = pd.DataFrame()
# specify pairs
div_pairs = [('f01', 'f02'), ('e04', 'e05'), ('e02', 'e15')]
# iterate
for pair in div_pairs:
    # division feature: added by '1e-6' to avoid `0/0` case
    div_train = np.divide((train_set[pair[0]]+1e-6), (train_set[pair[1]]+1e-6))
    div_test = np.divide((test_set[pair[0]]+1e-6), (test_set[pair[1]]+1e-6))
    # name the series
    div_train = div_train.rename(f'{pair[0]}_{pair[1]}')
    div_test = div_test.rename(f'{pair[0]}_{pair[1]}')
    # append data to the initilised dataframes
    df_train_div = pd.concat([df_train_div, pd.DataFrame(div_train)], axis=1, sort=False)
    df_test_div = pd.concat([df_test_div, pd.DataFrame(div_test)], axis=1, sort=False)

# create a metadata for these features
var_idx_div =  df_train_div.columns.tolist()
print(f'Division features: {var_idx_div}')

Division features: ['f01_f02', 'e04_e05', 'e02_e15']


Interaction among categorical pairs causes error in our training which we suspected it was due to some interactions in the val/test sets not exists in the train/design sets. Therefore, for this assignment, we try the interaction term between categorical variables (`['e21', 'b04', 'f09', 'e20', 'e11', 'a03']`) and non-categorical variables that attained the largest normalized coefficients, including target encoded hccv (`'f10'`) and a numeric variable (`'f02'`).

In [31]:
# specify the interaction pairs
for_pairs = ['e21', 'b04', 'f09', 'e20', 'e11', 'a03']
# create interaction pairs
interaction_pairs = []
for i in range(len(for_pairs)):
    for j in range(len(for_pairs)):
        if i < j:
            interaction_pairs.append(tuple([for_pairs[i], for_pairs[j]]))

In [32]:
print(f'Interaction pairs: {interaction_pairs}')

Interaction pairs: [('e21', 'b04'), ('e21', 'f09'), ('e21', 'e20'), ('e21', 'e11'), ('e21', 'a03'), ('b04', 'f09'), ('b04', 'e20'), ('b04', 'e11'), ('b04', 'a03'), ('f09', 'e20'), ('f09', 'e11'), ('f09', 'a03'), ('e20', 'e11'), ('e20', 'a03'), ('e11', 'a03')]


#### Create interaction terms in h2o
Since in Part 3 we decided to utilise the h2o built-in interaction terms creation in the `H2OGeneralizedLinearEstimator()` function, for the preprocessing this time, we have to create them with `h2o.interaction()` function.

In [33]:
# connect to h2o via localhost
h2o.init(ip="localhost", port=54323)

Checking whether there is an H2O instance running at http://localhost:54323 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "1.8.0_292"; OpenJDK Runtime Environment (build 1.8.0_292-8u292-b10-0ubuntu1~20.04-b10); OpenJDK 64-Bit Server VM (build 25.292-b10, mixed mode)
  Starting server from /opt/conda/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp1ovsexpy
  JVM stdout: /tmp/tmp1ovsexpy/h2o_jovyan_started_from_python.out
  JVM stderr: /tmp/tmp1ovsexpy/h2o_jovyan_started_from_python.err
  Server is running at http://127.0.0.1:54323
Connecting to H2O server at http://127.0.0.1:54323 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.2
H2O_cluster_version_age:,2 months and 16 days
H2O_cluster_name:,H2O_from_python_jovyan_kpe4fl
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,6.971 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"accepting new members, healthy"


In [34]:
# load data to h2o java vm
h2o_df_train = h2o.H2OFrame(train_set[for_pairs], destination_frame='df_train')
h2o_df_test = h2o.H2OFrame(test_set[for_pairs], destination_frame='df_test')

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [35]:
# generate interaction terms for train_set
h2o.interaction(h2o_df_train, 
                pairwise=True,
                factors=for_pairs,
                max_factors=1000,
                min_occurrence=1,
                destination_frame='train_interaction')

# generate interaction terms for test_set
h2o.interaction(h2o_df_test, 
                pairwise=True,
                factors=for_pairs,
                max_factors=1000,
                min_occurrence=1,
                destination_frame='test_interaction')


Interactions progress: |██████████████████████████████████████████████████| 100%
Interactions progress: |██████████████████████████████████████████████████| 100%


e21_b04,e21_f09,e21_e20,e21_e11,e21_a03,b04_f09,b04_e20,b04_e11,b04_a03,f09_e20,f09_e11,f09_a03,e20_e11,e20_a03,e11_a03
C_F,C_B,C_A4893,C_A,C_B,F_B,F_A4893,F_A,F_B,B_A4893,B_A,B_B,A4893_A,A4893_B,A_B
O_B,O_F,O_D53C5,O_B,O_H,B_F,B_D53C5,B_B,B_H,F_D53C5,F_B,F_H,D53C5_B,D53C5_H,B_H
O_B,O_B,O_D53C5,O_B,O_H,B_B,B_D53C5,B_B,B_H,B_D53C5,B_B,B_H,D53C5_B,D53C5_H,B_H
C_E,C_G,C_A4893,C_B,C_B,E_G,E_A4893,E_B,E_B,G_A4893,G_B,G_B,A4893_B,A4893_B,B_B
C_E,C_B,C_A4893,C_B,C_B,E_B,E_A4893,E_B,E_B,B_A4893,B_B,B_B,A4893_B,A4893_B,B_B
N_E,N_B,N_A4893,N_B,N_H,E_B,E_A4893,E_B,E_H,B_A4893,B_B,B_H,A4893_B,A4893_H,B_H
N_E,N_B,N_A4893,N_B,N_H,E_B,E_A4893,E_B,E_H,B_A4893,B_B,B_H,A4893_B,A4893_H,B_H
F_B,F_G,F_D913A,F_A,F_B,B_G,B_D913A,B_A,B_B,G_D913A,G_A,G_B,D913A_A,D913A_B,A_B
Q_F,Q_B,Q_A4893,Q_B,Q_H,F_B,F_A4893,F_B,F_H,B_A4893,B_B,B_H,A4893_B,A4893_H,B_H
Q_F,Q_B,Q_A4893,Q_B,Q_H,F_B,F_A4893,F_B,F_H,B_A4893,B_B,B_H,A4893_B,A4893_H,B_H


In [36]:
# convert h2o frames to pandas DataFrame
train_interaction = h2o.get_frame('train_interaction').as_data_frame()
test_interaction = h2o.get_frame('test_interaction').as_data_frame()

#### Concatenate all the preprocessed dataframes
Prepare dataframes for model training: train/val/test/design sets

Case 4 was used for our final prediction for Part 3

In [37]:
"""
Case 4: Add interaction terms (categorical variables)
-- based on variable importances with tree-based selection in Part 2.
"""
# exclude hccv from cat variables metadata
var_idx_cat_not_hccv = list(set(var_idx_cat) - set(var_idx_hccv))
# components = ['numeric and splines', 'categorical', 'hccv', 'division']
df_design_all = pd.concat([df_train_num, train_set[var_idx_cat_not_hccv], train_interaction, df_train_hccv,
                           df_train_div, train_set[var_idx_response]], axis=1, sort=False)
df_test_all = pd.concat([df_test_num, test_set[var_idx_cat_not_hccv], test_interaction, df_test_hccv,
                        df_test_div], axis=1, sort=False)

# there seem to be two rows with missing values in the design set caused through the interaction term
df_design_all = df_design_all.dropna(axis=0)

# split design into train/val sets
df_train_all, df_val_all = train_test_split(df_design_all, test_size=0.2, random_state=888)

#### Standardise numeric features

- Standardise train/val set for model selection

In [38]:
# get column indices for numerical features that need to standardise
# -- index intersection/union are used so we can apply the same function on all the Cases above
all_num_col = train_set[var_idx_num].columns.union(var_idx_spline).union(var_idx_div)
num_col = df_train_all.columns.intersection(all_num_col)

# create standardise pipeline
# -- parameters = List of (name, transformer, columns)
pipeline = ColumnTransformer([
    ('standardise', StandardScaler(), num_col)
])

# fit standardisation on the train set
train_scale = pipeline.fit_transform(df_train_all)
# apply the standardisation to the val set
val_scale = pipeline.transform(df_val_all)

# replace the values with scaled values
df_train_all[num_col] = train_scale
df_val_all[num_col] = val_scale

<ipython-input-38-5622a1ff2df5>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_all[num_col] = train_scale
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
<ipython-input-38-5622a1ff2df5>:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

- Standardise design/test set for test set prediction

In [39]:
# get column indices for numerical features that need to standardise
# -- index intersection/union are used so we can apply the same function on all the Cases above
all_num_col = train_set[var_idx_num].columns.union(var_idx_spline).union(var_idx_div)
num_col = df_design_all.columns.intersection(all_num_col)

# create standardise pipeline
# -- parameters = List of (name, transformer, columns)
pipeline = ColumnTransformer([
    ('standardise', StandardScaler(), num_col)
])

# fit standardisation on the design set
# -- need to remove the response during transform so the column index match with the test set
design_scale = pipeline.fit_transform(df_design_all.iloc[:,:-1])
# apply the standardisation to the test set
test_scale = pipeline.transform(df_test_all)

# replace the values with scaled values
df_design_all[num_col] = design_scale
df_test_all[num_col] = test_scale

### Vowpal Wabbit - Online Learning

- Adjust the code provided in the notebooks to create data for vw in the correct format.

In [54]:
# export preprocessed data to csv
df_train_all.to_csv(os.path.join(dirPData, 'train.csv'))
df_val_all.to_csv(os.path.join(dirPData, 'val.csv'))
df_design_all.to_csv(os.path.join(dirPData, 'design.csv'))
df_test_all.to_csv(os.path.join(dirPData, 'test.csv'))

Use loop to create data for vw; the `.csv` files will be removed after created `.vw` files to save spaces

In [56]:
# set file names
f_names = ['train', 'val', 'design', 'test']
for f_name in f_names:
    # setup parameters
    
    # label is in the last column
    label_index = 123
    # ignore column 0 which is index numbers
    ignore_columns = [0]
    # header exist
    skip_headers = True
    # True for all files except for `test.csv`
    if f_name == 'test':
        label_present = False
    else:
        label_present = True
    # iterate to create .vw files
    idx_row = 0
    # setup file paths
    csv_path = os.path.join(dirPData, f'{f_name}.csv')
    vw_path = os.path.join(dirPData, f'{f_name}.vw')
    with open(csv_path, 'r') as infile, \
         open(vw_path, 'w') as outfile:
        reader = csv.reader(infile)
        if skip_headers:
            headers = next(reader)
        for line in reader:
            idx_row += 1
            if label_present:
                label = line.pop(label_index)
            else:
                label = 1
            label = int(float(label))
            if label == 0:
                label = -1
            new_line = []
            new_line.append( "{} |n".format(label))
            for idx_col, item in enumerate(line):
                if idx_col in ignore_columns:
                    continue
                else:
                    categorical = False
                    try:
                        item_float = float(item)
                        if item_float == 0.0:
                            continue    # sparse format
                    except ValueError:
                        if item:
                            categorical = True
                        else:
                            continue

                    if categorical:
                        new_item =  "c{}_{}".format(idx_col + 1, item)
                    else:
                        new_item = "{}:{}".format(idx_col + 1, item )
                    new_line.append(new_item)
            new_line = " ".join( new_line )
            new_line += "\n"

            outfile.write(new_line)
    # remove file after finish writing
    os.remove(csv_path)
    # print to verify the removed file
    print(f'{csv_path} has been removed.')

../PData/train.csv has been removed.
../PData/val.csv has been removed.
../PData/design.csv has been removed.
../PData/test.csv has been removed.


Let's check the first line of each `.vw` file 

The data format for `vw` takes 1 for positive class and -1 for negative class.

For the test set, we noticed that in the code, 1 was assigned as a dummy lable when label does not exist.

In [57]:
for f_name in f_names:
    f_path = os.path.join(dirPData, f'{f_name}.vw')
    with open(f_path) as f:
        print(f_path + ':')
        print('--------------------')
        print(f.readline())

../PData/train.vw:
--------------------
-1 |n 2:-0.07964552000756642 3:0.5155896151647581 4:0.2754280848752945 5:-0.08559508295513285 6:-0.24875170031876934 7:0.18253989794928338 8:0.8291774496356544 9:-0.2506097557315126 10:0.11121335874123105 11:-0.0806597280741413 12:-0.0471813260628615 13:-0.07959543545878676 14:-1.5630780766640213 15:-0.2098414743395872 16:-0.256631940986384 17:0.19499434279462513 18:3.3644558799357234 19:0.36918470723810554 20:-0.10769034674949253 21:-0.19806986035078505 22:0.09929746756195929 23:-0.33187812977814507 24:-0.30517520359007255 25:-0.0844078250023573 26:0.9836573514444525 27:1.0084021975551538 28:1.0227884811276466 29:0.8794636206529151 30:-0.43115732291510916 32:1.524541759091392 33:1.624899154957833 34:1.754898604056127 35:2.1103193631537764 36:2.478390318934751 38:-1.3131986435507443 39:-1.2658449989889278 40:-0.9178095755360637 41:-0.6520502307639899 42:-0.3995208991068253 44:-0.18766456411536775 45:-0.28976869160655744 46:-0.6302357120284311 47:

### Model Training: Elastic Net

#### Hyperparameter tuning: hyperopt
To tune the hyperparameter for vowpal wabbit with hyperopt, we loaded the module from https://github.com/VowpalWabbit/vowpal_wabbit/blob/master/utl/vw-hyperopt.py

We downloaded the file and stored in the `PCode` directory.
- Hyperparameters tune ranges (algorithm = "Stochastic Gradient Descent" (SGD)): 
  - l2: [1e-8, 1e-4] --- Optional (LO)
  - l1: [1e-8, 1e-4] --- Optional (LO)
  - learning_rate: [0.01, 10]
  - passes: [1, 10] --- epoch
- loss function: logistic loss
- link: logistic
- random_seed = 888 --- hyperopt does not provide seed setting in the module; we have added seed to the command line part yet the results are not the same each time the code is run.
- bit = 18 is used

In [78]:
# tune the hyperparameter with hyperopt
!python vw-hyperopt.py --train ../PData/train.vw \
--holdout ../PData/val.vw --max_evals 200 --outer_loss_function logistic \
--vw_space ' --algorithms=sgd --l2=1e-8..1e-4~LO --l1=1e-8..1e-4~LO -l=0.01..10~L --passes=1..10~I --loss_function=logistic --link=logistic --random_seed=888 -b=18'

# # expected outcome
# vw  -d ../PData/train.vw -f ./current.model --holdout_off -c  --link logistic --loss_function logistic --passes 1 --random_seed 888 -b 18 -l 9.977644028429442 

2021-07-16 11:08:31,076 INFO     [root/vw-hyperopt:249]: loading true holdout class labels...
2021-07-16 11:08:31,549 INFO     [root/vw-hyperopt:260]: holdout length: 49999
2021-07-16 11:08:31,550 DEBUG    [root/vw-hyperopt:344]: starting hypersearch...
  0%|                                   | 0/200 [00:00<?, ?trial/s, best loss=?]2021-07-16 11:08:31,567 INFO     [hyperopt.tpe/tpe:873]: build_posterior_wrapper took 0.003958 seconds
2021-07-16 11:08:31,567 INFO     [hyperopt.tpe/tpe:909]: TPE using 0 trials
2021-07-16 11:08:31,569 INFO     [root/vw-hyperopt:315]: 

Starting trial no.1
2021-07-16 11:08:31,569 INFO     [root/vw-hyperopt:240]: executing the following command (training): vw  -d ../PData/train.vw -f ./current.model --holdout_off -c  --l1 1.839252732728083e-05 --l2 1.8328017739283907e-05 --link logistic --loss_function logistic --passes 10 --random_seed 888 -b 18 -l 6.767267767708859 
using l1 regularization = 1.83925e-05
using l2 regularization = 1.8328e-05
final_regressor 

only testing
predictions = ./holdout.pred
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using cache_file = ../PData/val.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.008391 0.008391            1            1.0   1.0000   0.7487      117
0.005359 0.002327            2            2.0   1.0000   0.7404      117
0.315627 0.625894            4            4.0  -1.0000   0.1269      117
0.527892 0.740158            8            8.0   1.0000   0.6310      117
0.659132 0.790372           16           16.0   1.0000   0.2784      117
0.898096 1.137061           32           32.0  -1.0000   0.2054      117
1.080178 1.262260           64           64.0  -1.0000   0.6076      117
0.931571 0.782964          128          128.0  -1.0000   0.3665      117
0.841142 0.750712          256          256.0   1.0000   0

2.370752 2.273486         2048         2048.0  -1.0000   0.0811      117
2.422702 2.474651         4096         4096.0   1.0000   0.9278      117
2.401679 2.380656         8192         8192.0  -1.0000   0.1223      117
2.366365 2.331052        16384        16384.0   1.0000   0.7139      117
2.364677 2.362989        32768        32768.0  -1.0000   0.0486      117

finished run
number of examples = 49999
weighted example sum = 49999.000000
weighted label sum = -853.000000
average loss = 2.366145
best constant = -0.017060
best constant's loss = 0.999709
total feature number = 5849387
2021-07-16 11:09:05,806 INFO     [root/vw-hyperopt:305]: parameter suffix: --l2 2.424204205571114e-08 --link logistic --loss_function logistic --passes 3 --random_seed 888 -b 18 -l 7.854537526703206 
2021-07-16 11:09:05,807 INFO     [root/vw-hyperopt:306]: loss value: 0.650065
2021-07-16 11:09:05,809 INFO     [root/vw-hyperopt:323]: evaluation time for this step: 0:00:03.004346
  2%|▏        | 5/200 [00:34<18

0.616914 0.595761        16384        16384.0   1.0000   0.5268      117
0.597293 0.577672        32768        32768.0   1.0000   0.5352      117
0.577540 0.557787        65536        65536.0   1.0000   0.8106      117
0.556031 0.534523       131072       131072.0  -1.0000   0.4838      117
0.536790 0.517548       262144       262144.0   1.0000   0.8220      117
0.520381 0.503971       524288       524288.0   1.0000   0.7593      117
0.508090 0.495799      1048576      1048576.0  -1.0000   0.6681      117

finished run
number of examples per pass = 199995
passes used = 6
weighted example sum = 1199970.000000
weighted label sum = -20898.000000
average loss = 0.506151
best constant = -0.034834
best constant's loss = 0.692995
total feature number = 140384214
2021-07-16 11:09:16,928 INFO     [root/vw-hyperopt:245]: executing the following command (validation): vw  -t -d ../PData/val.vw -i ./current.model -p ./holdout.pred --holdout_off -c 
only testing
predictions = ./holdout.pred
Num weig

using l1 regularization = 7.0486e-08
using l2 regularization = 6.63646e-06
final_regressor = ./current.model
Num weight bits = 18
learning rate = 0.0558393
initial_t = 0
power_t = 0.5
decay_learning_rate = 1
using cache_file = ../PData/train.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.693147 0.693147            1            1.0  -1.0000   0.5000      117
0.610789 0.528430            2            2.0  -1.0000   0.4105      117
0.620477 0.630166            4            4.0  -1.0000   0.4153      117
0.741906 0.863335            8            8.0   1.0000   0.5177      117
0.744583 0.747260           16           16.0   1.0000   0.4471      117
0.711872 0.679161           32           32.0  -1.0000   0.4130      117
0.679860 0.647847           64           64.0  -1.0000   0.3227      117
0.720875 0.761891          128  

...

2021-07-16 11:20:11,001 INFO     [root/vw-hyperopt:315]: 

Starting trial no.194
2021-07-16 11:20:11,004 INFO     [root/vw-hyperopt:240]: executing the following command (training): vw  -d ../PData/train.vw -f ./current.model --holdout_off -c  --l2 2.6800455908272362e-08 --link logistic --loss_function logistic --passes 1 --random_seed 888 -b 18 -l 0.13153577807217737 
using l2 regularization = 2.68005e-08
final_regressor = ./current.model
Num weight bits = 18
learning rate = 0.131536
initial_t = 0
power_t = 0.5
using cache_file = ../PData/train.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.693147 0.693147            1            1.0  -1.0000   0.5000      117
0.545414 0.397681            2            2.0  -1.0000   0.3281      117
0.560601 0.575788            4            4.0  -1.0000   0.3319      117
0.797324 1.034

2.144666 2.138511         1024         1024.0   1.0000   0.7555      117
2.019798 1.894931         2048         2048.0  -1.0000   0.1622      117
2.051307 2.082815         4096         4096.0   1.0000   0.9287      117
2.034556 2.017805         8192         8192.0  -1.0000   0.1355      117
2.010780 1.987005        16384        16384.0   1.0000   0.6223      117
2.000038 1.989296        32768        32768.0  -1.0000   0.0747      117

finished run
number of examples = 49999
weighted example sum = 49999.000000
weighted label sum = -853.000000
average loss = 1.996616
best constant = -0.017060
best constant's loss = 0.999709
total feature number = 5849387
2021-07-16 11:20:15,857 INFO     [root/vw-hyperopt:305]: parameter suffix: --link logistic --loss_function logistic --passes 2 --random_seed 888 -b 18 -l 1.096238550493158 
2021-07-16 11:20:15,858 INFO     [root/vw-hyperopt:306]: loss value: 0.653428
2021-07-16 11:20:15,860 INFO     [root/vw-hyperopt:323]: evaluation time for this step: 

0.616367 0.644529       262144       262144.0   1.0000   0.5618      117

finished run
number of examples per pass = 199995
passes used = 2
weighted example sum = 399990.000000
weighted label sum = -6966.000000
average loss = 0.640335
best constant = -0.034834
best constant's loss = 0.692995
total feature number = 46794738
2021-07-16 11:20:19,993 INFO     [root/vw-hyperopt:245]: executing the following command (validation): vw  -t -d ../PData/val.vw -i ./current.model -p ./holdout.pred --holdout_off -c 
only testing
predictions = ./holdout.pred
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using cache_file = ../PData/val.vw.cache
ignoring text input in favor of cache input
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
1.000000 1.000000            1            1.0   1.0000   0.5000      117
1.000000 1.000000            2            2.0   1.0000   0.5

0.352032 0.344109            4            4.0  -1.0000   0.0208      117
1.259927 2.167822            8            8.0   1.0000   0.0664      117
1.469184 1.678441           16           16.0   1.0000   0.9255      117
1.259514 1.049844           32           32.0  -1.0000   0.0153      117
1.104857 0.950200           64           64.0  -1.0000   0.0033      117
1.065441 1.026025          128          128.0  -1.0000   0.0299      117
1.042281 1.019121          256          256.0  -1.0000   0.1753      117
1.043032 1.043782          512          512.0   1.0000   0.6963      117
1.025670 1.008309         1024         1024.0  -1.0000   0.0237      117
0.927768 0.829867         2048         2048.0   1.0000   0.7591      117
0.832895 0.738021         4096         4096.0   1.0000   0.4354      117
0.770142 0.707390         8192         8192.0   1.0000   0.9607      117
0.710138 0.650133        16384        16384.0   1.0000   0.8898      117
0.667168 0.624199        32768        32768.0   1.0

2021-07-16 11:20:30,985 INFO     [root/vw-hyperopt:305]: parameter suffix: --l2 1.1171604187758875e-05 --link logistic --loss_function logistic --passes 2 --random_seed 888 -b 18 -l 6.779012122831262 
2021-07-16 11:20:30,985 INFO     [root/vw-hyperopt:306]: loss value: 0.673348
2021-07-16 11:20:30,987 INFO     [root/vw-hyperopt:323]: evaluation time for this step: 0:00:02.745868
100%|███████| 200/200 [11:59<00:00,  3.60s/trial, best loss: 0.6464570960668778]
2021-07-16 11:20:31,004 DEBUG    [root/vw-hyperopt:346]: the best hyperopt parameters: {'algorithm': 0, 'sgd_b': 0, 'sgd_l': 9.977644028429442, 'sgd_l1_outer': 0, 'sgd_l2_outer': 0, 'sgd_link': 0, 'sgd_loss_function': 0, 'sgd_passes': 1.0, 'sgd_random_seed': 0}
2021-07-16 11:20:31,008 INFO     [root/vw-hyperopt:349]: All the trials results are saved at ./trials.json
2021-07-16 11:20:31,009 INFO     [root/vw-hyperopt:353]: 

A full training command with the best hyperparameters: 
vw  -d ../PData/train.vw -f ./current.model --holdout

#### Best model Hyperparameters:

Even though in the log of the hyperopt h-p tuning, we noticed varieties of models including Ridge regression, Lasso Regression, Elastic Net Regression were fitted. The final model we obtained do not have any penalised coefficient and thus is a logistic regression. The learning rate tuned through hyperopt is `9.977644028429442` and the number of epoch (passes) used in the best model is `1`. The seed is set to `888`.

In [80]:
# fit the best hyperparameter on the design set
!vw  -d ../PData/design.vw -f ../PData/model.vw --holdout_off -c  --link logistic --loss_function logistic --passes 1 --random_seed 888 -b 18 -l 9.977644028429442 

final_regressor = ../PData/model.vw
Num weight bits = 18
learning rate = 9.97764
initial_t = 0
power_t = 0.5
creating cache_file = ../PData/design.vw.cache
Reading datafile = ../PData/design.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
0.693147 0.693147            1            1.0   1.0000   0.5000      117
0.350281 0.007414            2            2.0   1.0000   0.9926      117
1.787172 3.224063            4            4.0   1.0000   0.7460      117
1.939960 2.092748            8            8.0  -1.0000   0.7150      117
2.060769 2.181578           16           16.0   1.0000   0.0119      117
1.901272 1.741776           32           32.0   1.0000   0.0526      117
1.643163 1.385053           64           64.0   1.0000   0.8564      117
1.607780 1.572397          128          128.0  -1.0000   0.3009      117
1.558459 1.509137          256          256.0  -1.0000   0.9

#### Make prediction on the test set
As we have provided a dummy label (1) for the test set, the loss shown in the prediction result is meaningless because we do not possess labels. 

In [81]:
!vw -d ../PData/test.vw -t -i ../PData/model.vw -p ../PData/part5_pred.txt

only testing
predictions = ../PData/part5_pred.txt
Num weight bits = 18
learning rate = 0.5
initial_t = 0
power_t = 0.5
using no cache
Reading datafile = ../PData/test.vw
num sources = 1
average  since         example        example  current  current  current
loss     last          counter         weight    label  predict features
16.378593 16.378593            1            1.0   1.0000   0.0453      117
10.409762 4.440930            2            2.0   1.0000   0.9572      117
6.542706 2.675650            4            4.0   1.0000   0.5587      117
10.192379 13.842051            8            8.0   1.0000   0.0173      117
7.013160 3.833942           16           16.0   1.0000   0.7781      117
5.098898 3.184635           32           32.0   1.0000   0.1465      117
4.083545 3.068193           64           64.0   1.0000   0.5566      117
4.555026 5.026507          128          128.0   1.0000   0.7111      117
5.593367 6.631709          256          256.0   1.0000   0.5136      117
6.126

#### Create submission file

In [97]:
# get file path
f_path = os.path.join('..', 'PData', 'part5_pred.txt')
# initialise a list
model_pred_test = pd.read_csv(f_path, delimiter = '\n', header=None)

In [105]:
# create submission file
submission = pd.DataFrame({
    'unique_id': test_set['unique_id'],
    'Predicted': model_pred_test[0]
})
# save the submission file
f_name = 'Group_8_part5.csv'
f_path = os.path.join(dirPOutput, f_name)
submission.to_csv(f_path, index=False)

### Note on Part 5

#### Kaggle score: 0.83073
<img src="../screenshots/Group_8_part5_screenshot.png">

For Part 5, the logical processes for our solution are as follows:
- Applied the same preprocessing process as Part 3
- Trained the model through vowpal wabbit and performed h-p training with hyperopt
- Fitted the final vw model on the design set: model = Logistic Regression
- Predicted the test set and create the submission file for Kaggle competition

Compare the result to Part 3:

| Kaggle Competition   |        Part 3          |        Part 5         |
|:---------------------|:-----------------------|:----------------------|
|         AUC          |        0.84734         |       0.83073         |

For Part 5, we compared the result with Part 3 as they are both trained through a penalised logistic regression model. Apparently, the model we got from vowpal wabbit was a logistic regression with no penalised terms. Comparing the AUC score on the test set, we observed that online learning (Part 5) performed slightly worse than batch learning (Part 3). In our humble opinion, this result is in line with our expectations. We believed that batch learning would perform better than online learning as it possesses all available information while online learning only sees one at a time. Similar to human-being, we can make a firmer decision when seeing a big picture rather than just part of the picture. Nevertheless, an online learning framework like vowpal wabbit provides the benefit in terms of speed and memory efficiency. This would allow the system to train on huge datasets (i.e., billions of rows), which sometimes cannot fit in the machine memory.